# Market Market Simulation

We will make markets based on secret numbers assigned to each person.

**HOW TO RUN NOTEBOOK**: 
To run the simulation, simply click on ‘Run All’ under the ‘Runtime’ tab. Then, the simulation will give you your secret number and prompt you to input your name (you need to scroll to section titled **RUN SIMULATION**). After doing so, you will begin the rounds. For each round, input your quote as buy price and sell price in the format ‘buy@sell’ where buy is an integer and sell is an integer. Ideally, buy is less than sell since it doesn’t make sense to want to buy something for more than you want to sell it for, but the code lets you play around with the numbers however you want. At the end of the simulation, you will be given your opponent’s numbers and their strategies and your own PNL. 
You are free to change the constants in cell 2 as well as the strategy ranges of the computers in the strategy class section.


In [9]:
# import classes
import random
from time import sleep

In [10]:
# constants
# VARIATION ALLOWED: feel free to change any of these numbers
NUM_ROUNDS = 5
NUM_OPPONENTS = 5

MIN_SECRET = 1
MAX_SECRET = 20

## Player Class

Here we define the class for a player, both user and computers. Each player has the amount cash, stock, and settled pnl they own and also a secret number.

In [11]:
class Player:
    def __init__(self, name):
        self.id = name
        self.secret = random.randrange(MIN_SECRET, MAX_SECRET+1)

        # PNL
        self.cash = 0 # cash owned
        self.stock = 0 # stocks owned/short
        self.pnl = 0 # settled pnl

        # Transactions History
        self.history = []

    def getBalance(self):
        data = {"cash": self.cash, "stock": self.stock, "pnl": self.pnl}
        return f"\nPNL: {self.pnl} \ncash: {self.cash} \nstock: {self.stock}"

    def transact(self, price, qty, ctpy):
        self.history.append({"ctpy": ctpy, "price": price, "qty": qty})
        self.cash -= price*qty
        self.stock += qty
        if(self.stock == 0):
            self.pnl += self.cash
            self.cash = 0
        return True

In [12]:
# inherits the Player class, for only the user playing who serve as the market maker, not the computers
class MarketMaker(Player):
    def __init__(self, name):
        super().__init__(name)

## Strategy Classes
For each of the computers playing, they need a strategy to play with.

In [13]:
# Base Strategy Template
class StratTemplate(Player):
    def __init__(self, name):
        super().__init__(name)
        self.strat = random.randrange(1,4)
    
    def getSecret(self):
        return self.secret

    def getStrat(self):
        return self.strat
    
    def getAction(self):
        pass

# VARIATION ALLOWED: Feel free to change the margins of each play, but keep it the same on both the buy and sell side
def RandomizeStrat(fairValue, strat):
    # Plays Tight, Small Margins
    if strat == 1:
        return fairValue-1, fairValue+1
    # Plays Loose, Larger Margins
    elif strat == 2:
        return fairValue-3, fairValue+3
    # Plays Very Loose, Large Margins
    else:
        return fairValue-6, fairValue+6

class PlayerStrat(StratTemplate):
  # LIFT a market maker = buys at the ask price
  # HIT a market maker = sells at the bid price

  def getAction(self, bid, ask):
        fairValue = self.secret + 10*(NUM_OPPONENTS)
        buy, sell = RandomizeStrat(fairValue, self.strat)
        if buy >= ask:
          return "LIFT"
        elif sell <= bid:
          return "HIT"
        return "NA"

## Some Functions

In [14]:
def update_user(user: MarketMaker, response, bid, ask, ctpy):
    # Market Maker sells
    if(response == "LIFT"):
        return user.transact(ask, -1, ctpy)

    # Market Maker buys
    elif(response == "HIT"):
        return user.transact(bid, 1, ctpy)
    else:
        return

def update_computer(computer, response, bid, ask, ctpy):
    # Computer sells
    if(response == "HIT"):
        computer.transact(bid, -1, opp)
    
    # Computer buys
    elif(response == "LIFT"):
        computer.transact(ask, 1, opp)
    
    else:
        return

def Quote(quote):
    # FORMAT: buy@sell
    # Default amount 1
    try:
        bid, ask = quote.split("@")
        return int(bid), int(ask)
    except:
        print("Please reformat: bid@ask")
    return

def getRange(strat, fairValue):
    if strat == 1:
        return f"Buy at {fairValue - 1} or less, Sell at {fairValue+1} or more"
    elif strat == 2:
        return f"Buy at {fairValue - 3} or less, Sell at {fairValue+3} or more"
    elif strat == 3:
        return f"Buy at {fairValue - 6} or less, Sell at {fairValue+6} or more"

def getPNL(player):
  if(player.stock > 0):
    print(f"You have {player.stock} stocks")
    print(f"Exchanging your stock for cash at fair value...")
    player.cash += player.stock*fv
    player.stock = 0
    player.pnl += player.cash
    player.cash = 0
  if(player.stock < 0):
      print(f"You are short {player.stock} stock")
      print(f"Covering your short position...")
      player.cash += player.stock*fv
      player.stock = 0
      player.pnl += player.cash
      player.cash = 0
  return player.pnl, player.cash, player.stock

# RUN SIMULATION

You have a total of 5 opponents, for 6 players total. You will play for 5 rounds. Each player is assigned a secret number between 1 and 20 inclusive. Start and run the cell and make markets on the sum of all your numbers.

Variations: You can choose to change the number players, range of numbers, as well as the degree of risk each player is willing to take. To change these, the lines that are changeable are preceded by a ``# VARIATION ALLOWED`` comment.

In [15]:
player = MarketMaker(name="User")
opponents = [] 

for opp in range(NUM_OPPONENTS):
    opponents.append( PlayerStrat(f"{opp+1}") )

print(f"Welcome to this market making simulation! You will be a market maker amongst {NUM_OPPONENTS} market takers.")
print(f"Your secret number is: {player.secret}")
print(f"Make a market on: {player.secret} + SUM(all {NUM_OPPONENTS} opponents secret numbers)")
name = input("Input your name to start\n")

player.id = name

prompt = """Enter a quote in "bid@ask" format """
prompt2 = f"Make a market on: {player.secret} + SUM(all {NUM_OPPONENTS} opponents secret numbers)"
    
# Begin Game 
for i in range(1, NUM_ROUNDS+1):
    print("\n", "~"*50, f"Round {i}", "~"*50)
    quote = input(f"Enter a quote { player.id }>")
    bid, ask = Quote(quote)

    # Present offer to opponents
    for opp in opponents:
        # Get their action
        response = opp.getAction(bid,ask)
        # Transact
        update_user(player, response, bid, ask, opp)
        update_computer(opp, response, bid, ask, player)

        amt =  bid if(response == "HIT") else ask if response == "LIFT" else "--"
        
        if response != "NA":
            print(f"|\t Player {opp.id} {response} @ {amt} \t\t Current (avg) position: {opp.stock}@{opp.cash / (abs(opp.stock)) if opp.stock!=0 else 0}")
        else:
            print(f"|\t Player {opp.id} No Action \t\t Current (avg) position: {opp.stock}@{opp.cash / (abs(opp.stock)) if opp.stock!=0 else 0}")
    
    print("-" * 108)
        
    print("Your position:", player.getBalance())
    
## End Game
print("*"*20, "Statistics", "*"*20)
print("Revealing secret numbers and player ranges...")
sleep(1)
fv = player.secret
for opp in opponents:
    print(f"Player {opp.id}: {opp.secret}")
    opp_fv = int(opp.secret) + 10*NUM_OPPONENTS
    opp_range = getRange(opp.strat, opp_fv)
    print(f"Player Strategy: ", opp_range)
    fv += opp.secret
    sleep(0.5)
print(f"This means the fair value is:  {fv} ")
print("Computing your total pnl....")
sleep(2)
player.pnl, player.cash, player.stock = getPNL(player)
sleep(0.5)
print(f"Total pnl: >>>||  {player.pnl}  ||<<<")

Welcome to this market making simulation! You will be a market maker amongst 5 market takers.
Your secret number is: 18
Make a market on: 18 + SUM(all 5 opponents secret numbers)
Input your name to start
joanna

 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Round 1 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Enter a quote joanna>66@70
|	 Player 1 No Action 		 Current (avg) position: 0@0
|	 Player 2 HIT @ 66 		 Current (avg) position: -1@66.0
|	 Player 3 No Action 		 Current (avg) position: 0@0
|	 Player 4 HIT @ 66 		 Current (avg) position: -1@66.0
|	 Player 5 HIT @ 66 		 Current (avg) position: -1@66.0
------------------------------------------------------------------------------------------------------------
Your position: 
PNL: 0 
cash: -198 
stock: 3

 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Round 2 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Enter a quote joanna>62@73
|	 Player 1 No Action 		 Current (avg) position: 0@0
|	 Player 2 HIT @ 62 		 Curren